In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import main
from main import *
import multiprocessing
import concurrent.futures
import matplotlib.image as mpimg
import stat_funcs

In [2]:
%matplotlib inline
num_processes = multiprocessing.cpu_count()  # Get the number of CPU cores
inputs = range(5,10)  # Example input values
# sys.stdout.flush()
# with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
#     results = executor.map(initiate, inputs)

In [3]:
# %matplotlib inline
# delta = .1
# t = np.arange(-2*np.pi, 2*np.pi, delta)
# T1 = np.pi
# T2 = np.pi / 0.95
# signal = np.abs(2/len(t) * np.sin(2*np.pi/T1*t))*np.cos(2*np.pi/T2*t)
# freq = np.fft.rfftfreq(len(t),delta)
# plt.plot(freq,np.real(np.fft.rfft(signal)), label='real')
# plt.plot(freq, np.imag(np.fft.rfft(signal)), label='imag')
# plt.legend()
# plt.title('Example of fft of product of 2 sinusoids')
# plt.xlabel('Frequency')
# plt.ylabel('Amplitude in some units')
# %matplotlib inline
# plt.plot(t,signal)

In [4]:
from m_modules.energy import *

# for sep in np.arange(0.05, 5, 0.1):
#     print(f'at separation {sep} Bohr')
#     # since d Te / d S = d E_elec - Z_1 Z_2 / s^2, 
#     # d E_elec =  d Te / d S + Z_1 Z_2 / s^2
#     a,b,elec, tot = get_mol_energy_grad(7,7,sep)

In [14]:
def pol_fit(d, A, n, C):
    # here U is energy at separation of 1 Bohr
    # A = (1 / (C-U))** (1/n) - 1
    # A = 0
    return A / np.power(d,n) + C

def exp(freq, A, offset,n,C):
    return A*np.exp((freq-offset)*n) + C

        
min = 0.1
max = 10
stepsize = 0.1
atoms = np.array([8,6])
sep = np.arange(min, max+stepsize, stepsize)
data = gen_sep_data(atoms[0],atoms[1], min, max, stepsize) - np.prod(atoms) / sep**2 # gives total energy curve

# restricting data
from_ = 1
to_ = 9
data = data[(sep > from_) & (sep < to_)]
sep = sep[(sep > from_) & (sep < to_)]
exponent = 2.05
# fitting 
C = -np.sum(atoms**exponent)
U =  -289.201

# popt_, pcov_ = curve_fit(exp, sep, data,absolute_sigma=True,p0 = (-300, -0.5, -1.5,C-2),maxfev=100000)
# fit_ = exp(sep, *popt_)

popt_, pcov_ = curve_fit(pol_fit, sep, data,absolute_sigma=True,p0 = (-50, 1.8,C),maxfev=100000)
fit_ = pol_fit(sep, *popt_)
fitted_exponent = round(popt_[1],3)

d_i = 2.1
elec_equi, elec_grad, tot_grad = get_mol_energy_grad(atoms[0],atoms[1],d_i)
gradient = elec_grad[1] # d E / d sep --- keeping left atom fixed and shifting right atom

prediction = elec_equi + gradient * d_S_gamma(d_i, sep, fitted_exponent)

new_prediction = prediction
print(elec_equi)
get_values_uc(pol_fit, popt_, pcov_)
print(f" Unified energy = {pol_fit(0, *popt_)}")

# plotting
plt.scatter(sep, data, label = 'dft calcs')
plt.plot(sep, fit_, label=f'fit = A * d^(-{fitted_exponent}) + C, C = free atom elec energy')
plt.plot(sep, prediction, label='linearized pred')

plt.xlabel("Separation (Bohr); delta = 0.1 Bohr")
plt.ylabel("Energy (Hartree)")
plt.legend()

KeyboardInterrupt: 

In [6]:
print(sep[0])
def pol_fit(d, n,U, C):
    # here U is energy at separation of 1 Bohr
    A = (1 / (C-U))** (1/n) - 1
    return -1 / np.power(d+A,n) + C

pol_fit(1,2, -145, C)

1.1


-145.0